# 飞桨常规赛：量子电路合成 - 5月第2名方案

## 比赛介绍
量子电路合成是量子计算中十分重要的问题，对于实现量子计算机有指导意义。本次比赛以量子电路合成为题，旨在让参赛者了解并掌握如何使用给定量子门集合来高效的近似合成目标量子门，加深对量子计算的认识。  
  
详细介绍：[https://aistudio.baidu.com/aistudio/competition/detail/70](https://aistudio.baidu.com/aistudio/competition/detail/70)

### 赛题分析
> 比赛包含六道题目，其中前四题为基础题 (满分分别为 1 分，2 分，3分，10 分)，第五题为进阶题 (满分为 25 分)，第六题为挑战题 (满分为 60 分)  
  
从题目类型上看，可以分为两类:
- 给定结构求解电路参数(前三题)
- 不定结构分解电路以及参数求解(后三题)  

对于第一类，我们可以使用[量桨](https://qml.baidu.com/)搭建给定结构电路，并使用[飞桨](https://www.paddlepaddle.org.cn/)求解参数  
第二类难度提升，需要先求(cai)解(ce)出结构，然后使用第一类方法求解参数

## 解题思路  
对于第一类问题，参照量桨的[快速入门](http://https://qml.baidu.com/quick-start/quantum-neural-network.html)可以搭建出指定结构的量子电路，然后是求解参数。  
评分机制使用的是**量子门保真度**函数$F(U,V) = \left| {{\mathop{\rm Tr}\nolimits} \left( {U \times {V^T}} \right)} \right|/{2^n}$，于是可以使用$1 - F(U,V)$作为loss来优化参数
  
对于第二类问题，评分机制为$量子门保真度+量子电路复杂度$，问题变为**使用尽可能少的量子门搭建保真度高的量子电路**。  
对此，我联想到神经网络的万能近似定理，是不是也有一种量子电路结构可以近似任意量子电路呢？  
- 思路1: 仿照神经网络，使用控制门将所有的量子比特连接起来，形成网一样的结构  
![](https://ai-studio-static-online.cdn.bcebos.com/52899b5f2c714d249cb9b554bf56656e41831038b8584ce982488ff1dc37aa19)
  
- 思路2: 使用优化方法找到最优结构  
最先想到的是神经网络的方法，奈何旋转门和控制门正交，即单个控制门作用无法用旋转门替代，此路不通，尝试用其他方法  
第4题只有3个量子比特，可以使用暴力搜索的方法找到最优值，于是可以使用它来测试其他方法  
目前还没有探索出找到最优解的方法

## 具体方案

### 准备环境
- 安装量桨
- 解压试题
- 导入所需包

In [ ]:
!pip install paddle-quantum

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 81kB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 108.8MB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 62.1MB/s eta 0:00:01
  Created wheel for interval: filename=interval-1.0.0-cp37-none-any.whl size=14267 sha256=6a8496623cda1cb0222fa94d949120c29928f0b8ef50303291a71ca85c53eee6
  Stored in directory: /home/aistudio/.cache/pip/wheels/aa/fd/f0/28a65289ea3a10ea744cfe6d2393605383772be2ab2b75967e
Successfully built interval
  Found existing installation: networkx 2.4
    Uninstalling networkx-2.4:
      Successfully uninstalled networkx-2.4


In [ ]:
!unzip -oq /home/aistudio/data/data71784/飞桨常规赛：量子电路合成.zip -d ~/data/
!mv ~/data/飞桨常规赛：量子电路合成 ~/data/target

In [ ]:
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


### 基于量桨的量子电路类
为了方便计算损失函数、计算电路复杂度和输出结果，定义`MyCircuit`类

In [ ]:
class MyCircuit(UAnsatz):

    def gate_fidelity(self, target):
        a = self.U
        b = target
        a = paddle.cast(a, 'float64')
        a.stop_gradient = False
        b = paddle.cast(b, 'float64')
        b.stop_gradient = False
        c = paddle.fluid.layers.matmul(a, b, transpose_y=True)
        trace = paddle.trace(c)
        return paddle.abs(trace) / 2**self.n
    
    @property
    def complexity(self):
        c = 0
        for g, _, _ in self._UAnsatz__history:
            if g == 'u':
                c += 1
            elif g == 'CNOT':
                c += 8
        return c
    
    def output(self, path):
        history = []
        for g, w, p in self._UAnsatz__history:
            if g == 'ry':
                theta = p[0]
                if hasattr(p[0], 'numpy'):
                    theta = p[0].numpy()[0]                    
                history.append('R %d %f\n' % (w[0], np.mod(theta, 2 * np.pi)))
            elif g == 'CNOT':
                history.append('C %d %d\n' % tuple(w))
            else:
                raise OSError('未知的门`%s`' % g)
        with open(path, 'w') as f:
                for i in history:
                    f.write(i)

### 基于飞桨的自动优化器
使用飞桨动态图求解给定结构量子电路`circuit`参数，目标酉矩阵$U$定义为`target_tensor`

In [ ]:
class Net(paddle.nn.Layer):
    def __init__(self, n, p, dtype="float64"):
        super(Net, self).__init__()
        self.n = n
        self.theta = self.create_parameter(shape=[p], 
                                           default_initializer=paddle.nn.initializer.Uniform(low=-np.pi/2, high=np.pi/2),
                                           dtype=dtype, is_bias=False)
    def forward(self):
        cir = circuit(self.theta)
        loss = 1 - cir.gate_fidelity(target_tensor)
        return loss, cir

In [ ]:
class Optimizer:
    def __init__(self, net, learning_rate):
        scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=LR, T_max=100, eta_min=1e-3, last_epoch=99, verbose=False)
        opt = paddle.optimizer.Adam(learning_rate=scheduler, parameters=net.parameters())
        self.net = net
        self.scheduler = scheduler
        self.opt = opt

    def train(self, max_iters=600, log_iter=50, threshold=1e-8, display=True):
        last = 1
        for itr in range(1, max_iters + 1):
            loss, cir = self.net()
            loss.backward()
            self.opt.minimize(loss)
            self.opt.clear_grad()
            self.scheduler.step()
            if itr % log_iter == 0:
                l = loss.numpy()[0]
                if display:
                    print("iter:", "%3d" % itr, "  loss:", "%.4f" % l, "%.2e" % (l - last))
                if abs(l - last) < threshold:
                    break
                last = l
        return last

### 第一题 单量子比特门近似

In [ ]:
n = 1 #单量子比特电路
target = 1 / 2**0.5 * np.array([[1, -1], [1, 1]]) #目标酉矩阵
target_tensor = paddle.to_tensor(target) #转化为tensor
LR = 0.1 #学习率
ITR = 1000 #最大迭代次数

#定义电路结构
def circuit(theta):
    cir = MyCircuit(n)
    cir.ry(theta[0], 0) #单个旋转门
    return cir

In [ ]:
net = Net(n, 1) #定义网络
opt = Optimizer(net, LR) #定义自动优化器
opt.train(ITR, threshold=1e-15) #优化参数

theta_opt = net.theta.numpy() #优化好的参数
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta) #将优化好的参数加载到量子电路中
cir.output('Question_1_Answer.txt') #输出结果到文件

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:689: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


iter:  50   loss: 0.0075 -9.93e-01
iter: 100   loss: 0.0001 -7.34e-03
iter: 150   loss: 0.0000 -1.41e-04
iter: 200   loss: 0.0000 -6.12e-07
iter: 250   loss: 0.0000 -1.17e-10
iter: 300   loss: 0.0000 -5.40e-12
iter: 350   loss: 0.0000 -6.88e-15
iter: 400   loss: 0.0000 -1.11e-16
优化后的参数 theta:
 [1.57079633]


### 第二题 双量子比特门分解

In [ ]:
n = 2
target = np.loadtxt('/home/aistudio/data/target/Question_2_Unitary.txt') #读取目标实数酉矩阵U
target_tensor = paddle.to_tensor(target)
LR = 0.05
ITR = 1000

def circuit(theta):
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    cir.ry(theta[1], 1)
    cir.cnot([0, 1])
    cir.ry(theta[2], 0)
    cir.ry(theta[3], 1)
    return cir

In [ ]:
net = Net(n, 4)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_2_Answer.txt')

iter:  50   loss: 0.4692 -5.31e-01
iter: 100   loss: 0.0034 -4.66e-01
iter: 150   loss: 0.0000 -3.34e-03
iter: 200   loss: 0.0000 -1.82e-05
iter: 250   loss: 0.0000 -2.35e-06
iter: 300   loss: 0.0000 -8.36e-08
iter: 350   loss: 0.0000 -1.36e-09
iter: 400   loss: 0.0000 -2.64e-12
iter: 450   loss: 0.0000 -1.92e-13
iter: 500   loss: 0.0000 -3.33e-16
优化后的参数 theta:
 [-0.43582512  0.79511961 -2.79500593  0.42428991]


### 第三题 三量子比特门分解

In [ ]:
n = 3
target = np.loadtxt('/home/aistudio/data/target/Question_3_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.02
ITR = 1000

def circuit(theta):
    # 初始化 n 个量子比特的量子电路
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    cir.ry(theta[1], 1)
    cir.ry(theta[2], 2)
    cir.cnot([0, 1])
    cir.cnot([1, 2])
    cir.ry(theta[3], 0)
    cir.ry(theta[4], 2)
    return cir

In [ ]:
net = Net(n, 5)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_3_Answer.txt')

iter:  50   loss: 0.7453 -2.55e-01
iter: 100   loss: 0.2235 -5.22e-01
iter: 150   loss: 0.0351 -1.88e-01
iter: 200   loss: 0.0204 -1.48e-02
iter: 250   loss: 0.0120 -8.40e-03
iter: 300   loss: 0.0007 -1.12e-02
iter: 350   loss: 0.0000 -7.26e-04
iter: 400   loss: 0.0000 -9.48e-06
iter: 450   loss: 0.0000 -3.29e-06
iter: 500   loss: 0.0000 -2.76e-06
iter: 550   loss: 0.0000 -2.54e-08
iter: 600   loss: 0.0000 -6.69e-11
iter: 650   loss: 0.0000 -2.95e-11
iter: 700   loss: 0.0000 -1.66e-11
iter: 750   loss: 0.0000 1.33e-15
iter: 800   loss: 0.0000 -9.99e-16
优化后的参数 theta:
 [-2.30239562 -1.78154508 -0.33809543 -1.46190544  1.98867992]


### 可变量子电路类
为了便于尝试不同的电路结构，定义可变量子电路类  
给定控制门参数和位置，使用类似如下**旋转门-控制门-旋转门**的结构生成电路  
![](https://ai-studio-static-online.cdn.bcebos.com/abdaf603505e4e9e887bfbc2e4b64f7f18a265bdd27f45a1a316de8eecd599cd)


In [ ]:
class VariableCircuit(MyCircuit):
    def __init__(self, n, theta, controls=[]):
        super().__init__(n)
        self.controls = controls
        self.init(theta)

    def init(self, theta):
        offset = 0
        for i in range(self.n):
            self.ry(theta[offset+i], i)
        offset += self.n

        for a, b in self.controls:
            self.cnot([a, b])
            self.ry(theta[offset+0], a)
            self.ry(theta[offset+1], b)
            offset += 2
        self.params_size = offset

### 第四题 三量子比特门无结构分解
基于思路2，目前还没有找到又快又准的搜索方法

In [ ]:
n = 3
target = np.loadtxt('/home/aistudio/data/target/Question_4_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 1
ITR = 5000

In [ ]:
def circuit(theta):
    conts = [(1,2), (1,2), (0,1), (0,2), (0,1)]
    cir = VariableCircuit(n, theta, conts)
    return cir

net = Net(n, 15)
cir = circuit(net.theta)
print(cir.complexity)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-20)

40
iter:  50   loss: 0.3471 -6.53e-01
iter: 100   loss: 0.0580 -2.89e-01
iter: 150   loss: 0.0522 -5.81e-03
iter: 200   loss: 0.0522 -2.59e-05
iter: 250   loss: 0.0522 -1.65e-08
iter: 300   loss: 0.0526 4.19e-04
iter: 350   loss: 0.0522 -4.14e-04
iter: 400   loss: 0.0522 -4.38e-06
iter: 450   loss: 0.0522 -3.28e-08
iter: 500   loss: 0.0522 -3.15e-09
iter: 550   loss: 0.0522 -4.36e-11
iter: 600   loss: 0.0522 -8.84e-14
iter: 650   loss: 0.0522 -1.11e-16
iter: 700   loss: 0.0522 3.30e-07
iter: 750   loss: 0.0527 4.96e-04
iter: 800   loss: 0.0522 -4.96e-04
iter: 850   loss: 0.0522 -6.84e-07
iter: 900   loss: 0.0522 -6.58e-08
iter: 950   loss: 0.0522 -1.64e-11
iter: 1000   loss: 0.0522 -2.92e-12
iter: 1050   loss: 0.0522 -3.33e-16
iter: 1100   loss: 0.0522 1.58e-13
iter: 1150   loss: 0.0530 8.01e-04
iter: 1200   loss: 0.0522 -8.00e-04
iter: 1250   loss: 0.0522 -6.29e-07
iter: 1300   loss: 0.0522 -9.96e-09
iter: 1350   loss: 0.0522 -9.61e-10
iter: 1400   loss: 0.0522 -2.74e-13
iter: 1450   

0.05219339151955149

In [ ]:
theta_opt = net.theta.numpy()
theta_opt = np.mod(theta_opt, 2 * np.pi)
print("优化后的参数 theta:\n", theta_opt / np.pi)

cir = circuit(net.theta)
cir.output('Question_4_Answer.txt')

优化后的参数 theta:
 [3.42269108e-01 4.99999811e-01 4.99999904e-01 1.77098571e+00
 1.77098562e+00 1.92208134e+00 6.02804621e-01 1.34248397e+00
 1.06320314e-08 4.75030599e-08 6.93689542e-01 2.40332592e-01
 7.11078059e-01 4.51877705e-01 1.65428654e+00]


### 第五题 四量子比特门无结构分解
基于思路1，网状结构添加控制门  

In [ ]:
n = 4
target = np.loadtxt('/home/aistudio/data/target/Question_5_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.1
ITR = 1000

def circuit(theta):
    conts = [(1, 2)] #开头(1, 2)和结尾(3, 0)为尝试过程中最优结构的开头和结尾
    for i in range(7):
        for j in range(i%2, n, 2):
            conts.append((j%n, (j+1)%n))
    conts.append((3, 0))
    cir = VariableCircuit(n, theta, conts)
    return cir

In [ ]:
while 1: #循环运行直至满足及格条件
    net = Net(n, 36)
    opt = Optimizer(net, LR)
    l = opt.train(ITR)
    if l < 0.3:
        break

iter:  50   loss: 0.5351 -4.65e-01
iter: 100   loss: 0.4121 -1.23e-01
iter: 150   loss: 0.4038 -8.28e-03
iter: 200   loss: 0.4016 -2.17e-03
iter: 250   loss: 0.3978 -3.86e-03
iter: 300   loss: 0.3492 -4.86e-02
iter: 350   loss: 0.3282 -2.10e-02


In [ ]:
theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_5_Answer.txt')

In [ ]:
theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_6_Answer.txt')

### 打包结果

In [ ]:
!zip Answer.zip Question_?_Answer.txt